<a href="https://colab.research.google.com/github/anupshukla3639/apnacode_demo/blob/main/Movie_Reviewe_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
# positive-> 1
# negative-> 0
df.replace({"sentiment":{"positive": 1, "negative": 0}}, inplace= True)

<ipython-input-8-2529042971>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment":{"positive": 1, "negative": 0}}, inplace= True)


In [ ]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [ ]:
df["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
# LSTM-> LONG SHORT TERM MemoryError
# RNN-> TEXTUAL DATA


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# divide data->
train_df, test_df =train_test_split(df,test_size=0.2, random_state=42)

In [ ]:
# checking shape of data->
train_df.shape, test_df.shape

((40000, 2), (10000, 2))

In [ ]:
# convert text data in to word data->
tokenizer =Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_df["review"])

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df["review"]), maxlen=200)

In [ ]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [ ]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [ ]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]



In [ ]:
y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [ ]:
# Buiding LSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim= 128, input_length = 200))
model.add(LSTM(128, dropout= 0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation= "sigmoid"))
model.build(input_shape=(None, 200))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile model
model.compile(optimizer = "adam" , loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit(X_train, y_train, batch_size= 64 ,  epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 400ms/step - accuracy: 0.7165 - loss: 0.5394 - val_accuracy: 0.8269 - val_loss: 0.3868
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 198s 396ms/step - accuracy: 0.8571 - loss: 0.3486 - val_accuracy: 0.8572 - val_loss: 0.3513
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 394ms/step - accuracy: 0.8442 - loss: 0.3596 - val_accuracy: 0.8544 - val_loss: 0.3341
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 405ms/step - accuracy: 0.8851 - loss: 0.2743 - val_accuracy: 0.8745 - val_loss: 0.2912
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 390ms/step - accuracy: 0.9173 - loss: 0.2100 - val_accuracy: 0.8838 - val_loss: 0.2857


In [ ]:
model.save("model.h5")

In [ ]:
import joblib
joblib.dump(tokenizer, "tokenier.pkl")

['tokenier.pkl']

In [ ]:
# checking loss and accuracy of model
Loss, accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 54ms/step - accuracy: 0.8875 - loss: 0.2816


In [ ]:
print(Loss)

0.27458512783050537


In [ ]:
print(accuracy)

0.8903999924659729


In [ ]:
# Building predictive System

In [ ]:
def predictive_system(review):
 sequences = tokenizer.texts_to_sequences([review])
 padded_sequence =  pad_sequences(sequences, maxlen=200)
 prediction = model.predict(padded_sequence)
 sentiment = "positive"if prediction[0][0]>0.5 else "negative"
 return sentiment

In [ ]:
predictive_system("This movie is awesome")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


'positive'

In [ ]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


'positive'

In [ ]:
predictive_system("This movie is worst")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


'negative'